# Histopathologic Cancer Detection: Model Architecture

In this notebook, we'll design, implement, and train models for the Histopathologic Cancer Detection task. We'll explore different architectures, preprocessing techniques, and training strategies to develop effective models for identifying metastatic cancer in histopathology images.

## 1. Setup and Data Loading

First, let's import the necessary libraries and load the dataset.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm.notebook import tqdm
import cv2
from PIL import Image

# TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0, VGG16, MobileNetV2

# Scikit-learn
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Set plot style
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("TensorFlow version:", tf.__version__)

In [ ]:
# Set paths to the dataset
BASE_DIR = '../data'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
TEST_DIR = os.path.join(BASE_DIR, 'test')
TRAIN_LABELS_PATH = os.path.join(BASE_DIR, 'train_labels.csv')

# Load the training labels
try:
    train_labels = pd.read_csv(TRAIN_LABELS_PATH)
    print(f"Successfully loaded training labels with shape: {train_labels.shape}")
    print("\nSample of training labels:")
    display(train_labels.head())
    
    # Check class distribution
    class_distribution = train_labels['label'].value_counts().sort_index()
    print("\nClass distribution:")
    for label, count in class_distribution.items():
        print(f"Class {label} ({'Normal' if label == 0 else 'Metastatic Cancer'}): {count} ({count/len(train_labels)*100:.2f}%)")
except Exception as e:
    print(f"Error loading training labels: {e}")
    print("Please ensure the dataset is downloaded and the paths are correctly set.")

## 2. Data Preprocessing and Augmentation

Before we design our models, let's establish our data preprocessing and augmentation pipeline. Proper preprocessing is crucial for achieving good performance in deep learning models, especially for medical imaging tasks.

### 2.1 Image Loading and Normalization

First, we need to load the images and normalize their pixel values. Normalization helps the model converge faster and achieve better performance by ensuring that all input features are on a similar scale.

In [ ]:
def load_and_preprocess_image(image_id, directory, target_size=(96, 96), normalize=True):
    """Load and preprocess an image from the specified directory"""
    try:
        img_path = os.path.join(directory, f"{image_id}.tif")
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        
        # Resize if needed
        if img.shape[:2] != target_size:
            img = cv2.resize(img, target_size)
        
        # Normalize pixel values to [0, 1]
        if normalize:
            img = img.astype(np.float32) / 255.0
        
        return img
    except Exception as e:
        print(f"Error loading image {image_id}: {e}")
        return None

### 2.2 Train/Validation/Test Split

We'll split our data into training, validation, and test sets. The training set is used to train the model, the validation set is used to tune hyperparameters and monitor performance during training, and the test set is used for final evaluation.

In [ ]:
# Split the data into training and validation sets
train_df, val_df = train_test_split(
    train_labels, 
    test_size=0.2,  # 20% for validation
    random_state=42,
    stratify=train_labels['label']  # Ensure class balance in both sets
)

print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")

# Check class distribution in the splits
print("\nClass distribution in training set:")
train_class_dist = train_df['label'].value_counts(normalize=True) * 100
for label, percentage in train_class_dist.items():
    print(f"Class {label}: {percentage:.2f}%")

print("\nClass distribution in validation set:")
val_class_dist = val_df['label'].value_counts(normalize=True) * 100
for label, percentage in val_class_dist.items():
    print(f"Class {label}: {percentage:.2f}%")

### 2.3 Data Augmentation

Data augmentation is a technique to artificially expand the training dataset by creating modified versions of the existing images. This helps prevent overfitting and improves the model's ability to generalize to new data.

For histopathology images, appropriate augmentation techniques include:

1. **Rotation**: Cells and tissue structures can appear in any orientation
2. **Flipping**: Horizontal and vertical flips don't change the medical interpretation
3. **Slight zooming**: Simulates variations in magnification
4. **Brightness/contrast adjustments**: Accounts for staining variations
5. **Slight shifts**: Helps the model focus on the relevant features regardless of their exact position

We'll use Keras' ImageDataGenerator to apply these augmentations during training.

In [ ]:
# Define data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=90,  # Random rotations up to 90 degrees
    width_shift_range=0.1,  # Random horizontal shifts
    height_shift_range=0.1,  # Random vertical shifts
    shear_range=0.1,  # Shear transformations
    zoom_range=0.1,  # Random zooming
    horizontal_flip=True,  # Random horizontal flips
    vertical_flip=True,  # Random vertical flips
    fill_mode='nearest',  # Strategy for filling in newly created pixels
    brightness_range=[0.9, 1.1]  # Random brightness adjustments
)

# For validation, we only need to normalize the pixel values
val_datagen = ImageDataGenerator(
    rescale=1./255
)

### 2.4 Visualize Augmented Images

Let's visualize some examples of augmented images to ensure our augmentation pipeline is working as expected.

In [ ]:
def visualize_augmentations(image_id, directory, datagen, num_augmentations=5):
    """Visualize augmentations applied to a single image"""
    # Load the original image
    img = load_and_preprocess_image(image_id, directory, normalize=False)
    if img is None:
        return
    
    # Reshape for the data generator (batch_size=1)
    img = np.expand_dims(img, axis=0)
    
    # Create an iterator for the augmented images
    aug_iter = datagen.flow(img, batch_size=1)
    
    # Plot the original and augmented images
    plt.figure(figsize=(15, 4))
    
    # Original image
    plt.subplot(1, num_augmentations + 1, 1)
    plt.imshow(img[0].astype(np.uint8))
    plt.title('Original')
    plt.axis('off')
    
    # Augmented images
    for i in range(num_augmentations):
        aug_img = next(aug_iter)[0]
        plt.subplot(1, num_augmentations + 1, i + 2)
        plt.imshow((aug_img * 255).astype(np.uint8))
        plt.title(f'Augmentation {i+1}')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize augmentations for a normal tissue sample
normal_sample = train_df[train_df['label'] == 0].sample(1)['id'].values[0]
print(f"Augmentations for Normal Tissue Sample (ID: {normal_sample})")
visualize_augmentations(normal_sample, TRAIN_DIR, train_datagen)

# Visualize augmentations for a cancer tissue sample
cancer_sample = train_df[train_df['label'] == 1].sample(1)['id'].values[0]
print(f"Augmentations for Cancer Tissue Sample (ID: {cancer_sample})")
visualize_augmentations(cancer_sample, TRAIN_DIR, train_datagen)

### 2.5 Data Generators

Now, let's create data generators for training and validation. These generators will load and preprocess images in batches, which is more memory-efficient than loading the entire dataset at once.

In [ ]:
def create_data_generators(train_df, val_df, train_dir, batch_size=32):
    """Create data generators for training and validation"""
    # Training generator with augmentation
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_dir,
        x_col='id',
        y_col='label',
        target_size=(96, 96),
        batch_size=batch_size,
        class_mode='binary',
        validate_filenames=False,  # Skip validation for speed
        shuffle=True,
        seed=42
    )
    
    # Validation generator without augmentation
    val_generator = val_datagen.flow_from_dataframe(
        dataframe=val_df,
        directory=train_dir,
        x_col='id',
        y_col='label',
        target_size=(96, 96),
        batch_size=batch_size,
        class_mode='binary',
        validate_filenames=False,  # Skip validation for speed
        shuffle=False
    )
    
    return train_generator, val_generator

In [ ]:
# Create data generators
batch_size = 32
train_generator, val_generator = create_data_generators(train_df, val_df, TRAIN_DIR, batch_size)

## 3. Model Architectures

Now, let's design and implement different model architectures for the histopathologic cancer detection task. We'll explore three main approaches:

1. Custom CNN from scratch
2. Transfer learning with pre-trained models
3. Vision transformers

For each approach, we'll discuss its strengths, weaknesses, and suitability for histopathology image analysis.

### 3.1 Custom CNN from Scratch

Building a custom CNN from scratch gives us full control over the architecture and allows us to tailor it specifically to our task. This approach is useful when we have domain-specific knowledge about the problem or when pre-trained models might not be suitable.

**Strengths:**
- Full control over architecture design
- Can be tailored specifically for histopathology images
- Potentially smaller and faster than pre-trained models
- Better interpretability due to simpler architecture

**Weaknesses:**
- Requires more data to train effectively
- May not capture complex patterns as well as deeper pre-trained models
- Requires more hyperparameter tuning
- May take longer to converge

**Why it might work for histopathology:**
- Histopathology images have specific characteristics (cell structures, tissue patterns) that a custom CNN can be designed to detect
- The task is relatively focused (binary classification of a specific type of cancer)
- The image size is manageable (96x96 pixels)

In [ ]:
def create_custom_cnn(input_shape=(96, 96, 3)):
    """Create a custom CNN model for histopathology image classification"""
    model = Sequential([
        # First convolutional block
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Dropout(0.25),
        
        # Second convolutional block
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Dropout(0.25),
        
        # Third convolutional block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Dropout(0.25),
        
        # Fourth convolutional block
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Dropout(0.25),
        
        # Flatten and dense layers
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    return model

# Create and compile the custom CNN model
custom_cnn = create_custom_cnn()
custom_cnn.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Display model summary
custom_cnn.summary()

**Architecture Explanation:**

Our custom CNN consists of four convolutional blocks, each followed by max pooling, batch normalization, and dropout:

1. **Convolutional Blocks**: Each block contains two convolutional layers with increasing filter sizes (32 → 64 → 128 → 256). This allows the network to learn increasingly complex features.
   
2. **Max Pooling**: Reduces spatial dimensions, making the network more computationally efficient and helping it focus on the most important features.
   
3. **Batch Normalization**: Normalizes the activations of the previous layer, which helps with faster convergence and reduces the risk of overfitting.
   
4. **Dropout**: Randomly sets a fraction of input units to 0 during training, which helps prevent overfitting.
   
5. **Dense Layers**: After flattening, we have a dense layer with 512 units, followed by the output layer with a sigmoid activation function for binary classification.

This architecture is designed to capture both low-level features (like edges and textures) and high-level features (like cell structures and tissue patterns) in histopathology images.

### 3.2 Transfer Learning with Pre-trained Models

Transfer learning involves using a pre-trained model (trained on a large dataset like ImageNet) as a starting point and fine-tuning it for our specific task. This approach leverages the knowledge learned from a large and diverse dataset, which can be beneficial even for specialized tasks like histopathology image analysis.

**Strengths:**
- Leverages knowledge from pre-training on large datasets
- Requires less data to achieve good performance
- Often converges faster during training
- Can capture complex patterns and features

**Weaknesses:**
- Pre-trained models are typically trained on natural images, which differ from histopathology images
- Larger models with more parameters, which can lead to overfitting
- Higher computational requirements
- Less interpretable due to complex architectures

**Why it might work for histopathology:**
- Despite differences between natural and histopathology images, low-level features (edges, textures) are still relevant
- Pre-trained models have learned robust feature representations that can generalize to new domains
- Fine-tuning allows the model to adapt to the specific characteristics of histopathology images

We'll implement transfer learning with three popular architectures: ResNet50, EfficientNetB0, and MobileNetV2.

In [ ]:
def create_transfer_learning_model(base_model_name, input_shape=(96, 96, 3), trainable=False):
    """Create a transfer learning model using a pre-trained base model"""
    # Select the base model
    if base_model_name == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'efficientnet':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'mobilenet':
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError(f"Unsupported base model: {base_model_name}")
    
    # Freeze the base model if not trainable
    base_model.trainable = trainable
    
    # Create the model
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    
    return model

In [ ]:
# Create and compile the ResNet50 model
resnet_model = create_transfer_learning_model('resnet50', trainable=False)
resnet_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Display model summary
print("ResNet50 Transfer Learning Model:")
resnet_model.summary()

In [ ]:
# Create and compile the EfficientNetB0 model
efficientnet_model = create_transfer_learning_model('efficientnet', trainable=False)
efficientnet_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Display model summary
print("EfficientNetB0 Transfer Learning Model:")
efficientnet_model.summary()

In [ ]:
# Create and compile the MobileNetV2 model
mobilenet_model = create_transfer_learning_model('mobilenet', trainable=False)
mobilenet_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Display model summary
print("MobileNetV2 Transfer Learning Model:")
mobilenet_model.summary()

**Architecture Explanation:**

For our transfer learning models, we're using pre-trained networks (ResNet50, EfficientNetB0, and MobileNetV2) as feature extractors. Here's how our architecture works:

1. **Pre-trained Base Model**: We use the convolutional layers of the pre-trained model (without the top classification layers) to extract features from our images. Initially, we freeze these layers to preserve the learned features.

2. **Global Average Pooling**: This reduces the spatial dimensions of the feature maps, resulting in a fixed-size feature vector regardless of input size.

3. **Custom Top Layers**: We add our own fully connected layers on top of the base model:
   - A dropout layer to prevent overfitting
   - A dense layer with ReLU activation to learn task-specific features
   - Batch normalization to stabilize training
   - Another dropout layer
   - A final output layer with sigmoid activation for binary classification

This approach allows us to leverage the powerful feature extraction capabilities of pre-trained models while adapting them to our specific task of histopathologic cancer detection.